# Miembros de Grupos

* PRODUCTOS
  * autor registrante
     * Institución
     * Grupo


El resultado completo del análisis se encuentra en la siguiente carpeta del Google Drive:
https://drive.google.com/drive/u/0/folders/1d3UJK5QU5Y6XVB7NxnicGgnefDsfIiu-

## Modules, functions and data

In [1]:
import wosplus as wp

In [2]:
import pandas as pd
import numpy as np
import time
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

internal functions

In [4]:
def iloc_0(df):
    if not df.empty:
        df=df.iloc[0]
    else:
        df=pd.Series(dtype=str)
    return df
def get_author(RH,other=False,
               document_type={'C':"cédula", 'P':"passport", 'E':"cédula de extranería", 'T':"tarjeta de identidad"},
              roles={"ORI":"Advisor"}):
    if RH.get("TXT_TOTAL_NAMES"):
        aud={
            "full_name":str(RH.get("TXT_TOTAL_NAMES")),
            "first_names": str(RH.get("TXT_NAMES_RH")),
            "first_last_name": str(RH.get("TXT_PRIM_APELL")), #DEBUG: last_names
            "second_last_name": str(RH.get("TXT_SEG_APELL")), 
            "intials":''.join([s[0] for s in str(RH.get("TXT_NAMES_RH")).split()]),
            "aliases":[str(RH.get("TXT_CITACION_BIBLIO"))],
        }
    else:
        aud={}
    if RH.get("TPO_SEXO"):
        aud["sex"]=str(RH.get("TPO_SEXO"))
    if RH.get("TPO_NACIONALIDAD"):  
        aud["nacionality"]=str(RH.get("TPO_NACIONALIDAD")) #privado
    if RH.get("TPO_ESTADO_CIVIL"):
        aud["marital_status"]=str(RH.get("TPO_ESTADO_CIVIL")) #privado
    if RH.get("DTA_NACIM"):
        aud["date_of_birth"]=str(RH.get("DTA_NACIM")) #privado
    
    if RH.get("SGL_PAIS_NACIM"):
        PS=ps[ps['SGL_PAIS']==RH.get("SGL_PAIS_NACIM")]
        PS=iloc_0(PS)
        aud["country_of_birth"]=str(PS.get("TXT_NME_PAIS")) #privado
    if RH.get("COD_MUN_NACIM"):
        MP=mp[ (mp["COD_RH_MUNICIPIO"]==RH.get("COD_RH_MUN_NACIM")) & ( mp["COD_MUNICIPIO"]==RH.get("COD_MUN_NACIM"))]
        MP=iloc_0(MP)
        aud["town_of_birth"]=str(MP.get("TXT_NME_MUNICIPIO"))
    if RH.get("COD_RH"):
        aud["external_ids"]=[
                         {"value":str(RH.get("COD_RH")),
                          "source":"cvlac"
                         }]
    
    if RH.get("COD_ORCID") and not pd.isna(RH.get("COD_ORCID")):
        aud["external_ids"].append({"value":RH.get("COD_ORCID"),
                                    "source":"orcid"
                                   })
    if RH.get("AUTOR_ID_SCP") and not pd.isna(RH.get("AUTOR_ID_SCP")):
        aud["external_ids"].append({"value":RH.get("AUTOR_ID_SCP"),
                                    "source":"orcid"
                                   })
    if RH.get("NRO_DOCUMENTO_IDENT") and not pd.isna(RH.get("NRO_DOCUMENTO_IDENT")):
        aud["document"]=[{"value":RH.get("NRO_DOCUMENTO_IDENT"),
                      "type":document_type.get( RH.get("TPO_DOCUMENTO_IDENT") )}]

        if not pd.isna(RH.get("NRO_PASAPORTE")) and "passport" in [d.get("type") for d in aud["document"] ] :
            aud["document"].append({"value":RH.get("NRO_PASAPORTE"),
                                "type":"passport"
                               })
    if not other:
        aud["registrant"]=[{"value":True,
                              "source":"scienti"}] #DEBUG Unificar autor de correspondencia, envio preprint
        aud["role"]="author"
    else:
        aud["registrant"]=[{"value":False,
                              "source":"scienti"}] #DEBUG Unificar autor de correspondencia, envio preprint
        aud["role"]=roles.get(str(RH.get("TPO_PARTICIPACION")))
    return aud        

## Load the CvLAC and GrupLAC tables

In [6]:
dtype={
'COD_RH':str,
'NRO_ID_GRUPO':str,
'COD_RH_MUNICIPIO':str,
'COD_INST':str,
'COD_INST_AVALA':str,
'COD_INST_MACRO':str,
'COD_INSTITUCION':str,         
'COD_INST_OTRO':str,
'ID_DEPARTAMENTO':str,
'COD_DEPA_DANE': str,
'COD_MUNI_DANE':str,
'COD_RH_OTRO':str,
'COD_RH_COAUTOR':str
 }
#converters={'COD_INST_AVALA':lambda s:'0' if pd.isna(s) else str(s)},

In [7]:
sc=wp.wosplus('drive.cfg')

In [8]:
cv=sc.read_drive_excel('EN_PRODUCTO.xlsx',dtype=dtype)

In [9]:
cv.shape

(342440, 58)

In [10]:
rh=sc.read_drive_excel('EN_RECURSO_HUMANO.xlsx',dtype=dtype)

In [11]:
tp=sc.read_drive_excel('EN_TIPO_PRODUCTO.xlsx',dtype=dtype)

In [12]:
gr=sc.read_drive_excel('GRUPLAC_EN_PRODUCTO_GR.xlsx',dtype=dtype)

In [13]:
PPC=sc.read_drive_excel('RE_PRODUCTO_PALABRA_CLA.xlsx',dtype=dtype)
PC=sc.read_drive_excel('EN_PALABRA_CLAVE.xlsx',dtype=dtype)

In [14]:
prh=sc.read_drive_excel('RE_PRODUCTO_RECURSO_HUM_OTRO.xlsx',dtype=dtype)

In [15]:
ps=sc.read_drive_excel('EN_PAIS.xlsx',dtype=dtype)
dp=sc.read_drive_excel('EN_DEPARTAMENTO.xlsx',dtype=dtype)
mp=sc.read_drive_excel('EN_MUNICIPIO.xlsx',dtype=dtype)

In [16]:
psa=sc.read_drive_excel('RE_PRODUCTO_SECTOR_APL.xlsx',dtype=dtype)

In [17]:
sa=sc.read_drive_excel('EN_SECTOR_APLICACION.xlsx',dtype=dtype)

In [18]:
gi=sc.read_drive_excel('GRUPLAC_RE_GRUPO_INSTITUCION.xlsx',dtype=dtype)

In [19]:
it=sc.read_drive_excel('EN_INSTITUCION.xlsx',dtype=dtype)

In [20]:
GR=sc.read_drive_excel('gruplac_grupo_pesquisa.xlsx',dtype=dtype)

In [21]:
ac=sc.read_drive_excel('EN_AREA_CONOCIMIENTO.xlsx',dtype=dtype)

In [22]:
prg=sc.read_drive_excel('GRUPLAC_EN_PROGRAMA_COLCIENCIAS.xlsx',dtype=dtype)

## JSON Generation code

Sample at https://jsonblob.com/api/918880355558375424/

To include `authors.affiliations.branches.descriptions` set `OPTIMIZE_STORAGE=False`

In [23]:
OPTIMIZE_STORAGE=True
if OPTIMIZE_STORAGE:
    borrar='_BORRAR'
else:
    borrar=''
prs=[]
#i=888#117294#0#340454#666#

prmax=10
#if True:
for i in cv.index:#range(340469,cv.index.max()+1):#
    #if i==101:#20:
    #    break
    pr={}
    if i%100==0:
        print(str(i).zfill(6),end='\r')    
    cvi=cv.loc[i]
    DEBUG=False
    #Tipo de Producto
    TP=tp[tp['COD_TIPO_PRODUCTO']==cvi.get('COD_TIPO_PRODUCTO')]
    TP=iloc_0(TP)

    #author
    RH=rh[rh["COD_RH"]==cvi.get("COD_RH")]
    RH=iloc_0(RH)

    #coauthor/advisor
    PRH=prh[(prh['COD_RH']==cvi['COD_RH']) & (prh['COD_PRODUCTO']==cvi['COD_PRODUCTO'])]
    PRH=iloc_0(PRH)

    #Sector Aplicación
    PSA=psa[ (psa['COD_RH']==cvi['COD_RH']) & (psa['COD_PRODUCTO']==cvi['COD_PRODUCTO']) ]
    PSA=iloc_0(PSA)

    SA=sa[sa['COD_SECTOR_APLICACION']==PSA.get('COD_SECTOR_APLICACION')]
    SA=iloc_0(SA)

    #Product in Group
    GRR=gr[ (gr['COD_RH']==cvi['COD_RH']) & (gr['COD_PRODUCTO']==cvi['COD_PRODUCTO']) ]
    GRR=iloc_0(GRR)
    pr["year_published"]=GRR.get('NRO_ANO_PRESENTA')
    pr["month_published"]=GRR.get('NRO_MES_PRESENTA')

    #Group code number o
    GI=gi[gi['NRO_ID_GRUPO']==GRR.get('NRO_ID_GRUPO')]
    GI=iloc_0(GI)

    #Institute 
    IT=it[it['COD_INST']==GI.get('COD_INST')]
    IT=iloc_0(IT)

    #Group information
    GP=GR[GR['NRO_ID_GRUPO']==GRR.get('NRO_ID_GRUPO')]
    GP=iloc_0(GP)

    DP=dp[dp['ID_DEPARTAMENTO']==IT.get('SGL_DEPARTAMENTO')]
    DP=iloc_0(DP)
    PS=ps[ps['SGL_PAIS']==DP.get('SGL_PAIS')]
    PS=iloc_0(PS)


    GRP=gr[(gr['COD_RH']==cvi.get('COD_RH')) & (gr['COD_PRODUCTO']==cvi.get('COD_PRODUCTO'))]
    GRP=iloc_0(GRP)

    GRU=GR[GR["NRO_ID_GRUPO"]==GRP.get("NRO_ID_GRUPO")]
    GRU=iloc_0(GRU)

    PRG=prg[prg['COD_PROGRAMA']==GRU.get('COD_PROGRAMA')]
    PRG=iloc_0(PRG)

    AC=ac[ac["COD_AREA_CONOCIMIENTO"]==GRU.get("COD_AREA_CONHEC")]
    AC=iloc_0(AC)

    if DEBUG: print('general metadata')

    pr['source_checked']=[]
    pr['source_checked'].append(
        {'source': 'Scienti',
         'COD_RH': cvi.get('COD_RH'),
         'COD_PRODUCTO':cvi.get('COD_PRODUCTO')
        })

    pr['title']=cvi.get('TXT_NME_PROD')

    if not pd.isna( cvi.get('TXT_RESUMEN_PROD') ):
        pr['abstract']=cvi.get(f'TXT_RESUMEN_PROD{borrar}')
    else:
        pr['abstract']=''

    if not pd.isna( cvi.get('TXT_OBSERV_PROD') ):
        pr['notes']=cvi.get(f'TXT_OBSERV_PROD{borrar}')
    else:
        pr['notes']=''

    pr["lang"]=str(cvi.get("SGL_IDIOMA"))

    pr['product_subtype']=[{'title':TP.get('TXT_NME_TIPO_PRODUCTO')}]
    if not pd.isna(cvi.get('SGL_CATEGORIA')):
        pr['product_subtype'][0]['SGL_CATEGORIA']=cvi.get('SGL_CATEGORIA')
    else:
        pr['product_subtype'][0]['SGL_CATEGORIA']=''  
    #TODO → ADD GRUPLAC PRODUCT information
    #TODO → Add specific 'product_subtype' information

    if DEBUG: print('topics')

    tp_names=[]
    stop=0
    while not TP.empty:
        TP=tp[tp['COD_TIPO_PRODUCTO']==TP.get('COD_TIPO_PRODUCTO_PADRE')]
        if not TP.empty:
            TP=TP.iloc[0]
        else:
            TP=pd.Series(dtype=str)
        if not pd.isna(TP.get('TXT_NME_TIPO_PRODUCTO')):
            tp_names.append(TP.get('TXT_NME_TIPO_PRODUCTO'))
            tp_names.reverse()
        stop+=1
        if stop>10:
            break

    tp_list=[]
    for ii in range(len(tp_names)):
        tp_list.append( {'title':tp_names[ii],
                         'level':ii})

    pr["product_type"]=tp_list

    pc=PPC[(PPC['COD_RH']==cvi.get('COD_RH')) & (PPC['COD_PRODUCTO']==cvi.get('COD_PRODUCTO'))]

    lpc=[]
    for ii in pc['COD_PALABRA_CLAVE'].values: 
        lpc.append( PC[(PC['COD_RH']==cvi.get('COD_RH')) & (PC['COD_PALABRA_CLAVE']==ii)].iloc[0].to_dict())

    pr["keywords"]=[d.get('TXT_NME_PALABRA_CLAVE') for d in lpc]

    if DEBUG: print('general authors metadata')

    pr["authors"]=[]


    pr["authors"].append(get_author(RH))


    if not pd.isna(PRH.get("COD_RH_COAUTOR")) and len(PRH.get("COD_RH_COAUTOR"))==10:
        RH=rh[rh["COD_RH"]==PRH.get("COD_RH_COAUTOR")]
        RH=iloc_0(RH)
        if RH.empty:
            RH["COD_RH"]=PRH["COD_RH"]
        #    RH["TXT_NAMES_RH"]=''
        RH["TPO_PARTICIPACION"]=PRH["TPO_PARTICIPACION"]
        pr["authors"].append(get_author(RH,other=True))

    pr["profile_description"]=RH.get(f"TXT_DESC_PERFIL{borrar}")


    if DEBUG: print('sector de aplicación')
    isa=True
    if SA.empty:
        isa=False
    sal=[]
    stop=0
    while isa:
        stop+=1
        if SA.get('TXT_NME_SECTOR_APLIC'):
            sal.append(SA.get('TXT_NME_SECTOR_APLIC'))
        else:
            isa=False
        if SA.get('COD_SECTOR_APLICACION') and not SA.get('COD_SECTOR_APLICACION')!=SA.get('COD_SECT_APLIC_PADRE'):
            isa=False
        else:
            SA=sa[sa['COD_SECTOR_APLICACION']==SA.get('COD_SECT_APLIC_PADRE')]
            SA=iloc_0(SA)
        if stop>10:
            print("to many PADREs in COD_SECT_APLIC_PADRE")
            break
    sal.reverse()        

    sall=[]
    for ii in range(len(sal)):
        sall.append({"title":sal[ii],"level":ii})

    pr["application_sector"]=sall


    if DEBUG: print('affiliation university')

    if IT.empty:
        aff={}
    else:
        aff={
        "name":str(IT.get('NME_INST')),
        'acronym':str(IT.get('SGL_INST')),
        'addresses': [{'line_1': str(IT.get("TXT_DIRECCION")),
                       "telephone":str(IT.get("TXT_TELEFONO")),
                       "email":str(IT.get("TXT_EMAIL"))
                                       }],
        'city': str(IT.get("TXT_CIUDAD_INST")), #town
        'date_constitution': str(IT.get("DTA_CONSTITUCION")),                  
        'external_urls': [{'source': 'scienti',
                                     'url': str(IT.get("URL_HOME_PAGE"))}],
                         # EN_INSTITUCION_OTRA → EN_INSTITUCION
        'external_ids': [{'source': 'NIT', 'value': str(IT.get("TXT_NIT"))}],
        'state': str(DP.get("TXT_NME_DEPARTAMENTO")),
        'state_code': {"value":str(DP.get("COD_DEPA_DANE")),
                       "type":"DANE"}, #https://www.fopep.gov.co/wp-content/uploads/2019/02/Tabla-C%C3%B3digos-Dane.pdf
        'country': PS.get(str("TXT_NME_PAIS"))   
        }

    if DEBUG: print('affiliation group')

    if GRU.empty:
        branch={}
    else:
        branch={
        "name":str(GRU.get("NME_GRUPO")),                                      
        "type":"group",
        'external_urls': [{ # → NRO_ID_GRUPO
        'source': 'scienti',
        'url': f'https://scienti.colciencias.gov.co/gruplac/jsp/visualiza/visualizagr.jsp?nro={GRU.get("NRO_ID_GRUPO")}'
                        }],
        'external_ids': [{'source': 'gruplac', 'id': str(GRU.get("COD_ID_GRUPO"))}],                                
        "formation_year":GRU.get("ANO_FORMACAO"),
        "month_formation_year":GRU.get("MES_FORMACAO"),
        #'subjects': [{'source': 'area_ocde','subjects': ['Ciencias Sociales']},
        #                                     {'source': 'subarea_ocde', 'subjects': ['Ciencias Políticas']},
        #                                     {'source': 'sub_subarea_ocde', 'subjects': ['Ciencias Políticas']},
        #                                    ],
        "classifications":[{"classification":str(GRU.get("TXT_CLASIF")),
                                                    "begin_date":str(GRU.get("DTA_CLASIF")),
                                                    "end_date":str(GRU.get("DTA_FIN_CLASIF")),
                                                    "source":"scienti"
                                                   }],
        "descriptions":{
            'PLAN_TRABAJO': GRU.get(f"TXT_PLAN_TRABAJO{borrar}"),
            'ESTADO_ARTE': GRU.get(f"TXT_ESTADO_ARTE{borrar}"),
            'OBJETIVOS': GRU.get(f"TXT_OBJETIVOS{borrar}"),
            'RETOS': GRU.get(f"TXT_RETOS{borrar}"),
            'VISION': GRU.get(f"TXT_VISION{borrar}"),
        },
        'programa_minciencias': {'programa':PRG.get("NME_PROGRAMA"),
                            'subdirección':PRG.get("NME_SUBDIRECCION")}    
        }

    if DEBUG: print('áreas ocde')

    if not AC.empty:
        subjects=[]
        stop=0 #Emergency stop
        STOP=False
        if AC.empty:
            STOP=True
        while not STOP:
            subjects.append({'title':AC.get("TXT_NME_AREA"),
                            'level':AC.get("NRO_NIVEL"),
                            'source': 'area_ocde'})    

            if AC.get('NRO_NIVEL')==0:
                STOP=True
            else:
                AC=ac[ac["COD_AREA_CONOCIMIENTO"]==AC.get("COD_AREA_PADRE")]
                AC=iloc_0(AC)

            #Emergency stop
            stop+=stop
            if stop>10:
                print("EMERGENCY EXIT in Area de Conocimiento")
                break
        if branch:
            branch['subjects']=subjects

    if aff:
        aff['branches']=[branch]
        pr['authors'][0]['affiliations']=[aff]
    else:
        pr['authors'][0]['affiliations']=[]

    prs.append(pr)
    if i%1000==0:
        #print(f'save at {i}')
        f=open('products.json','w',encoding='utf8')
        json.dump(prs,f,cls=NpEncoder,ensure_ascii=False)
        f.close()

     #TODO → Second author/advisor affiliation        
f=open('products.json','w',encoding='utf8')
json.dump(prs,f,cls=NpEncoder,ensure_ascii=False)
f.close()

In [24]:
len(prs)

342440

## Extract researcher information from JSON

and export to Excel

In [25]:
prs=pd.DataFrame(prs)

In [26]:
def to_excel(df):
    iv=pd.DataFrame()

    iv['title']=df['title']
    iv['year_published']=df['year_published']#.astype(int)
    iv['month_published']=df['month_published']#.astype(int)

    iv['COD_RH']=df['source_checked'].str[0].str['COD_RH']
    iv['COD_PRODUCTO']=df['source_checked'].str[0].str['COD_PRODUCTO']
    iv['product_type']=df['product_type'].str[0].str['title']
    iv['product_subtype']=df['product_subtype'].str[0].str['title']
    iv["SGL_CATEGORIA"]=df['product_subtype'].str[0].str["SGL_CATEGORIA"]

    iv['full_name']=df['authors'].str[0].str['full_name']
    #iv[]=df['authors'].str[0].str['external_ids'].str[0].str['value'] 
    iv['identification_type']=df['authors'].str[0].str['document'].str[0].str['type']
    iv['identification']=df['authors'].str[0].str['document'].str[0].str['value']

    iv['institution']=df['authors'].str[0].str['affiliations'].str[0].str['name']
    iv['institution_NIT']=df['authors'].str[0].str['affiliations'].str[0].str['external_ids'].str[0].str['value']

    iv['group']=df['authors'].str[0].str['affiliations'].str[0].str['branches'].str[0].str['name']
    iv['GroupLAC_id']=df['authors'].str[0].str['affiliations'].str[0].str['branches'].str[0].str['external_ids'].str[0].str['id']
    try:    
        iv['AREA_OCDE']=df['authors'].str[0].str['affiliations'].str[0].str['branches'].str[0].str['subjects'].apply(
             lambda l: [d for d in l if d.get('level')==0]).str[0].str['title']
        iv['SUB_AREA_OCDE']=df['authors'].str[0].str['affiliations'].str[0].str['branches'].str[0].str['subjects'].apply(
             lambda l: [d for d in l if d.get('level')==1]).str[0].str['title']
        iv['SUB_SUB_AREA_OCDE']=df['authors'].str[0].str['affiliations'].str[0].str['branches'].str[0].str['subjects'].apply(
             lambda l: [d for d in l if d.get('level')==2]).apply(
             lambda l: [d for d in l] if len(l)>0 else [{'title':''}]).str[0].str['title']
    except TypeError:
        iv['AREA_OCDE']=''
        iv['SUB_AREA_OCDE']=''
        iv['SUB_SUB_AREA_OCDE']=''
    iv['programa_minciencias']=df['authors'].str[0].str['affiliations'].str[0].str['branches'].str[0].str['programa_minciencias'].str['programa']
    iv['group_classification']=df['authors'].str[0].str['affiliations'].str[0].str['branches'].str[0].str['classifications'].str[0].str['classification']
    iv['group_classification_start']=df['authors'].str[0].str['affiliations'].str[0].str['branches'].str[0].str['classifications'].str[0].str['begin_date']
    iv['group_classification_end']=df['authors'].str[0].str['affiliations'].str[0].str['branches'].str[0].str['classifications'].str[0].str['end_date']
    try:
        iv['PLAN_TRABAJO']=df['authors'].str[0].str['affiliations'].str[0].str['branches'].str[0].str["descriptions"].str['PLAN_TRABAJO']
    except:
        iv['PLAN_TRABAJO']=''
    try:
        iv['ESTADO_ARTE'] =df['authors'].str[0].str['affiliations'].str[0].str['branches'].str[0].str["descriptions"].str['ESTADO_ARTE']
    except:
        iv['ESTADO_ARTE'] =''
    try:
        iv['OBJETIVOS']   =df['authors'].str['affiliations'].str[0].str['branches'].str[0].str[0].str["descriptions"].str['OBJETIVOS']
    except:
        iv['OBJETIVOS']   =''
    try:
        iv['RETOS']       =df['authors'].str['affiliations'].str[0].str['branches'].str[0].str[0].str["descriptions"].str['RETOS']
    except:
        iv['RETOS']       =''
    try:
        iv['VISION']      =df['authors'].str['affiliations'].str[0].str['branches'].str[0].str[0].str["descriptions"].str['VISION'] 
    except:
        iv['VISION']      =''
    return iv

In [27]:
iv=to_excel(prs)

In [28]:
iv=iv.dropna(subset=['GroupLAC_id']).reset_index(drop=True)

In [29]:
iv=iv.drop_duplicates(['identification','GroupLAC_id']).reset_index(drop=True)

In [30]:
iv.to_excel('investigadores.xlsx',index=False)

##  Appendix

Some checks

In [53]:
ne=pd.read_excel('/home/restrepo/Downloads/Coordinadores que no se encontraron_AA (1).xlsx')

In [54]:
iv[iv['identification']=='42986946']

,title,year_published,month_published,COD_RH,COD_PRODUCTO,product_type,product_subtype,SGL_CATEGORIA,full_name,identification_type,identification,institution,institution_NIT,group,GroupLAC_id,AREA_OCDE,SUB_AREA_OCDE,SUB_SUB_AREA_OCDE,programa_minciencias,group_classification,group_classification_start,group_classification_end,PLAN_TRABAJO,ESTADO_ARTE,OBJETIVOS,RETOS,VISION
858,Gestión integral de desechos Facultad de Medic...,2002.0,1.0,0000088293,5543,Producción técnica,Página web,GC_V-00,Luz Marina Restrepo Munera,cédula,42986946,IPS UNIVERSITARIA,811016192,Grupo Ingeniería de tejidos y terapias celulares,COL0027213,,,,"Ciencia, Tecnología e Innovación en Salud",B,2019-12-06 00:00:00,2021-12-05 00:00:00,NaN,NaN,,,


In [55]:
ne=ne.fillna(0).reset_index(drop=True).rename({'CEDULA':'identification'},axis='columns')
ne['identification']=ne['identification'].astype(str)

In [58]:
ne.merge(iv,on='identification',how='left')[
    ['COD_ID_GRUPO', 'Esatado', 'NOMBRE_COORDINADOR', 'identification',
       'title', 'year_published', 'month_published', 'COD_RH',
       'COD_PRODUCTO', 'product_type', 'product_subtype', 'SGL_CATEGORIA',
       'full_name', 'identification_type', 'institution', 'institution_NIT',
       'group', 'GroupLAC_id', 'AREA_OCDE', 'SUB_AREA_OCDE',
       'SUB_SUB_AREA_OCDE', 'programa_minciencias', 'group_classification',
       'group_classification_start', 'group_classification_end']
      ].to_excel('found.xlsx',index=False)

In [66]:
prs['authors'].str[1].dropna().iloc[2]

{'full_name': 'Fernando Cardeno Lopez',
 'first_names': 'Fernando',
 'first_last_name': 'Cardeno',
 'second_last_name': 'Lopez',
 'intials': 'F',
 'aliases': ['CARDENO LOPEZ FERNANDO'],
 'sex': 'M',
 'nacionality': 'C',
 'marital_status': 'U',
 'date_of_birth': '1974-02-03 00:00:00',
 'country_of_birth': 'Colombia',
 'town_of_birth': 'ITAGÜÍ',
 'external_ids': [{'value': '0000626554', 'source': 'cvlac'}],
 'document': [{'value': '98473797', 'type': 'cédula'}],
 'registrant': [{'value': False, 'source': 'scienti'}],
 'role': 'Advisor'}

In [80]:
prs[prs['year_published'].apply(lambda x: not pd.isna(x))
   ].sample(100).to_json('products_sample.json',orient='records')